In [1]:
!pip install --quiet optuna 
import optuna 
from optuna import trial
from catboost import CatBoostClassifier
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io, os
from sklearn.metrics import roc_auc_score , log_loss
from sklearn.model_selection import KFold

# set some variables
SEED = 12
FOLDS = 21

In [2]:
#encoding took place in the last notebook.
train = pd.read_csv('/kaggle/input/s4e10-nn/train.csv')
test = pd.read_csv('/kaggle/input/s4e10-nn/test.csv', index_col = 'id')

# define numerical columns
NUM_COLS =   ['person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'loanincomeratio', 'loanemploymentratio', 'intageincome', 'monthlyincome'] 

# define categorical columns
CAT_COLS =  ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file', 'riskflag'] 

# define cols for model
COLSET = NUM_COLS + CAT_COLS

#define the target column
TARGET = 'loan_status'

   cat_params={
    'task_type'           : "CPU",
    'loss_function'       : 'Logloss',
    'eval_metric'         : "AUC",
    'bagging_temperature' : 0.25,
    'colsample_bylevel'   : 0.40,
    'iterations'          : 5_000,
    'learning_rate'       : 0.045,
    'max_depth'           : 7,
    'l2_leaf_reg'         : 0.80,
    'min_data_in_leaf'    : 30,
    'random_strength'     : 0.25,
    'random_state'        : 42,
    'early_stopping_rounds': 200,
    'use_best_model'       : True,
    'allow_writing_files' : False,

In [3]:
def optimize_auc(trial):
    # define parameter test space
    cb_params = {
        'objective':'CrossEntropy',
        'iterations': 712,
        'learning_rate': trial.suggest_float('learning_rate', 0.07, 0.12),
        'max_depth' : trial.suggest_int('max_depth',4,10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 1.0),
        'random_strength': trial.suggest_float('random_strength',0.75, 2.0),
        #'depth': trial.suggest_int('depth', 6, 10),
 
        'min_data_in_leaf': 40,
        "use_best_model": True,
        "task_type": "GPU",
        'l2_leaf_reg': 0.8,
        'random_seed': 12
    }

#         cb_params = {
#             'iterations': 400,
#             'learning_rate': .001,
#             'l2_leaf_reg': 2,
#             'bagging_temperature': 1.2,
#             'random_strength': 1.4,
#             'depth': 4,
#             'min_data_in_leaf': 22,
#             "use_best_model": True,
#             "task_type": "GPU",
#             'random_seed': 12
#         }
 

    # variable to hold scores
    scores = np.zeros(FOLDS)
    # define KFold routine
    kf = KFold(n_splits = FOLDS, shuffle=True ,random_state=SEED)
    
    # KFold split, define Xs and Ys,
    for i, (trndex, valdex) in enumerate(kf.split(train.index)):
        xt,yt = train.loc[trndex,COLSET],train.loc[trndex,TARGET]        
        xv,yv = train.loc[valdex,COLSET],train.loc[valdex,TARGET]

        # build a model with the params and define the callback
        model=CatBoostClassifier(**cb_params, cat_features=CAT_COLS)
        # fit the model
        model.fit(xt,yt,
                  eval_set=(xv,yv), 
                  verbose=False, 
                  early_stopping_rounds=12,
                  use_best_model=True
                 )
 
        
        # make predictions
        ypred=pd.DataFrame(model.predict_proba(xv),columns=['o','l'])
        
        # score them, report fold efficacy, add to list
        ascore = roc_auc_score(yv,ypred['l'])
        print(f"Fold {i} score: {ascore}")
        scores[i] = ascore

    # output the mean of the fold scores
    return np.mean(scores)

In [4]:
# run the optuna study to find optimal hyperparameters
study = optuna.create_study(direction="maximize")
study.optimize(optimize_auc, n_trials=100)

#report the results 
print('Best parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2024-11-09 01:03:19,843] A new study created in memory with name: no-name-b086a9c8-6232-4f60-a5f5-be05b3d04f0e


Fold 0 score: 0.9497346445717418
Fold 1 score: 0.9567110215478083
Fold 2 score: 0.942253041946846
Fold 3 score: 0.9464153209109731
Fold 4 score: 0.9499593940346811
Fold 5 score: 0.951172603511112
Fold 6 score: 0.9579244524846832
Fold 7 score: 0.9530502629980752
Fold 8 score: 0.950210609765416
Fold 9 score: 0.9537239139530062
Fold 10 score: 0.9553186369778073
Fold 11 score: 0.9589322116836198
Fold 12 score: 0.9469495924659904
Fold 13 score: 0.9623316933123124
Fold 14 score: 0.9494100620745021
Fold 15 score: 0.9564957268309177
Fold 16 score: 0.9550642510385877
Fold 17 score: 0.9528766359431889
Fold 18 score: 0.9469989532889941
Fold 19 score: 0.9496464152660947


[I 2024-11-09 01:09:21,759] Trial 0 finished with value: 0.9521883304843252 and parameters: {'learning_rate': 0.07685872772537773, 'max_depth': 10, 'bagging_temperature': 0.34801551937995867, 'random_strength': 1.3119344066068521}. Best is trial 0 with value: 0.9521883304843252.


Fold 20 score: 0.9507754955644706
Fold 0 score: 0.9459795116444155
Fold 1 score: 0.9605629158818185
Fold 2 score: 0.9460825761820899
Fold 3 score: 0.9480848861283643
Fold 4 score: 0.9502543408162875
Fold 5 score: 0.9495360070363441
Fold 6 score: 0.9572968963015263
Fold 7 score: 0.950797640737424
Fold 8 score: 0.9514495141489009
Fold 9 score: 0.9548976761553276
Fold 10 score: 0.9533295367819966
Fold 11 score: 0.96029776468261
Fold 12 score: 0.9530980957477402
Fold 13 score: 0.965145066099651
Fold 14 score: 0.9464323947902858
Fold 15 score: 0.9531426410115295
Fold 16 score: 0.9574755586827275
Fold 17 score: 0.9517540263062944
Fold 18 score: 0.9506661073347153
Fold 19 score: 0.9564837915746532


[I 2024-11-09 01:12:05,839] Trial 1 finished with value: 0.9531589772066976 and parameters: {'learning_rate': 0.08555358929484212, 'max_depth': 4, 'bagging_temperature': 0.46776047947111354, 'random_strength': 1.6075753768032484}. Best is trial 1 with value: 0.9531589772066976.


...

Fold 20 score: 0.9538867032407086
Fold 0 score: 0.9515169642235457
Fold 1 score: 0.9588458961650304
Fold 2 score: 0.9480675965951543
Fold 3 score: 0.9495428571428571
Fold 4 score: 0.9529056117484699
Fold 5 score: 0.9532966063164627
Fold 6 score: 0.9607360506014453
Fold 7 score: 0.9551645410657856
Fold 8 score: 0.9537458451134713
Fold 9 score: 0.9575895586695017
Fold 10 score: 0.9543996305034389
Fold 11 score: 0.9625045549667357
Fold 12 score: 0.9560827550277897
Fold 13 score: 0.9663764384620619
Fold 14 score: 0.9493758554300515
Fold 15 score: 0.9552076538377909
Fold 16 score: 0.9596150155872312
Fold 17 score: 0.9551398956932463
Fold 18 score: 0.9523033751338935
Fold 19 score: 0.9561917364845501


[I 2024-11-09 06:06:35,507] Trial 98 finished with value: 0.9553748943157025 and parameters: {'learning_rate': 0.0854719353497061, 'max_depth': 6, 'bagging_temperature': 0.20668081219707707, 'random_strength': 1.5389455312537819}. Best is trial 84 with value: 0.9561892620012361.


Fold 20 score: 0.9542643418612412
Fold 0 score: 0.9521247921539399
Fold 1 score: 0.9621023582274885
Fold 2 score: 0.9495969086882272
Fold 3 score: 0.9496273291925467
Fold 4 score: 0.9541276502044924
Fold 5 score: 0.9524366276075281
Fold 6 score: 0.9583909508269263
Fold 7 score: 0.9520983231942686
Fold 8 score: 0.9534453719976567
Fold 9 score: 0.9573928410321092
Fold 10 score: 0.9550491765314586
Fold 11 score: 0.9632820747743338
Fold 12 score: 0.9562024811623467
Fold 13 score: 0.9679884927209197
Fold 14 score: 0.9524118007011096
Fold 15 score: 0.9560503917162441
Fold 16 score: 0.9579830695062583
Fold 17 score: 0.9564699708072293
Fold 18 score: 0.9536807979440478
Fold 19 score: 0.9578403937897202


[I 2024-11-09 06:09:13,627] Trial 99 finished with value: 0.9556724619551351 and parameters: {'learning_rate': 0.10282572698915765, 'max_depth': 5, 'bagging_temperature': 0.16699123046571912, 'random_strength': 1.7167185655167851}. Best is trial 84 with value: 0.9561892620012361.


Fold 20 score: 0.9508198982789853
Best parameters: {'learning_rate': 0.09030128590511562, 'max_depth': 6, 'bagging_temperature': 0.12657694966395555, 'random_strength': 1.4441256496986168}
Best score: 0.9561892620012361


In [5]:
# variables to hold oof, predictions, and scores
oof = np.zeros(len(train['loan_status']))
pres = np.zeros(len(test))
scores= np.zeros(FOLDS)

# define KFold routine
kf = KFold(n_splits = FOLDS, shuffle=True ,random_state=SEED)

# KFold split, define Xs and Ys,
for i, (trndex, valdex) in enumerate(kf.split(train.index)):
    xt,yt = train.loc[trndex,COLSET],train.loc[trndex,TARGET]        
    xv,yv = train.loc[valdex,COLSET],train.loc[valdex,TARGET]

    # build a model with the params and define the callback
    model = CatBoostClassifier( **study.best_trial.params,cat_features=CAT_COLS)

    # fit the model
    model.fit(xt,yt,
              eval_set=(xv,yv), 
              verbose=False, 
              early_stopping_rounds=12,
              use_best_model=True
             )

    # make oof predictions
    ypred=pd.DataFrame(model.predict_proba(xv),columns=['o','l'])
    oof[valdex] = ypred['l']

    # score them, report fold efficacy, add to list
    ascore = roc_auc_score(yv,ypred['l'])
    print(f"Fold {i} score: {ascore}")
    scores[i] = ascore

    # make test predictions
    tpred = pd.DataFrame(model.predict_proba(test[COLSET]), columns=['o','l'])
    pres += tpred['l']

# output the mean of the fold scores
print(f"AUC of Run: {np.mean(scores)}")

# Divide the testset predictions by the number of folds
pres = pres/FOLDS
    



Fold 0 score: 0.9515498079517731
Fold 1 score: 0.9624882053548566
Fold 2 score: 0.9494005330077916
Fold 3 score: 0.9507565217391305
Fold 4 score: 0.9544481915415276
Fold 5 score: 0.955203275234442
Fold 6 score: 0.9633963519585611
Fold 7 score: 0.954680336390493
Fold 8 score: 0.9562893201905198
Fold 9 score: 0.95729345977247
Fold 10 score: 0.9547487238292998
Fold 11 score: 0.9619773240741704
Fold 12 score: 0.955654814992786
Fold 13 score: 0.9686379894652963
Fold 14 score: 0.9523332521101487
Fold 15 score: 0.9590858167737955
Fold 16 score: 0.9601040102613134
Fold 17 score: 0.9569919145865451
Fold 18 score: 0.9498148909913616
Fold 19 score: 0.9558807167782067
Fold 20 score: 0.9531814329402615
AUC of Run: 0.956376994759274


In [6]:
oofer = train.loc[:,['loan_status']]
oofer['oof'] = oof
oofer.to_csv('/kaggle/working/oof.csv', index=True)

subm = pd.read_csv('/kaggle/input/playground-series-s4e10/sample_submission.csv')
subm['loan_status'] = pres
subm.to_csv('/kaggle/working/submission.csv', index=False)


In [7]:
!head /kaggle/working/oof.csv

,loan_status,oof
0,0,0.05355573145023094
1,0,0.008061428030321102
2,0,0.01659425069697953
3,0,0.013147971643722221
4,0,0.005391760154112165
5,0,0.006646155150856595
6,0,0.017850523674677
7,0,0.11694433041229772
8,0,0.19121191782819952


In [8]:
!head /kaggle/working/submission.csv

id,loan_status
58645,0.9994449407884206
58646,0.037442208292979534
58647,0.368434184812697
58648,0.010529454303255482
58649,0.09106914889952891
58650,0.9391615792220253
58651,0.0016547593106835893
58652,0.011121815190161869
58653,0.33364950597442883
